<a href="https://www.kaggle.com/code/rishabhbhartiya/2025-ai-job-trends-top-skills?scriptVersionId=243382951" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from collections import Counter, defaultdict
from IPython.display import display, clear_output

In [2]:
df = pd.read_csv("/kaggle/input/global-ai-job-market-and-salary-trends-2025/ai_job_dataset.csv")

In [3]:
df["application_deadline"] = pd.to_datetime(df["application_deadline"])
df["posting_date"] = pd.to_datetime(df["posting_date"])
df = df.drop(columns = "job_id")

In [4]:
skill_list = []
for skill in df["required_skills"]:
    skill_list.append(skill)
all_skills = []
for combo in skill_list:
    skills = [skill.strip() for skill in combo.split(',')]
    all_skills.extend(skills)
skill_counts = Counter(all_skills)
for skill, count in skill_counts.most_common():
    print(f"{skill}: {count}")

Python: 4450
SQL: 3407
TensorFlow: 3022
Kubernetes: 3009
Scala: 2794
PyTorch: 2777
Linux: 2705
Git: 2631
Java: 2578
GCP: 2442
Hadoop: 2419
Tableau: 2341
R: 2311
Computer Vision: 2284
Data Visualization: 2270
Deep Learning: 2189
MLOps: 2164
Spark: 2155
NLP: 2145
Azure: 2144
AWS: 2018
Mathematics: 1943
Docker: 1862
Statistics: 1833


In [5]:
skill_categories = {
    'NLP': 'AI/ML', 'Deep Learning': 'AI/ML', 'Computer Vision': 'AI/ML',
    'TensorFlow': 'AI/ML', 'PyTorch': 'AI/ML', 'MLOps': 'AI/ML',
    
    'SQL': 'Big Data', 'Hadoop': 'Big Data', 'Spark': 'Big Data', 'Scala': 'Big Data',
    
    'AWS': 'Cloud/DevOps', 'Azure': 'Cloud/DevOps', 'GCP': 'Cloud/DevOps',
    'Docker': 'Cloud/DevOps', 'Kubernetes': 'Cloud/DevOps', 'Git': 'Cloud/DevOps', 'Linux': 'Cloud/DevOps',

    'Python': 'Programming', 'Java': 'Programming', 'R': 'Programming',

    'Tableau': 'Visualization', 'Data Visualization': 'Visualization',

    'Statistics': 'Math/Stats', 'Mathematics': 'Math/Stats',
}

def categorize(skill):
    return skill_categories.get(skill, 'Other')

job_skill_counts = defaultdict(Counter)
for _, row in df.iterrows():
    job_title = row["job_title"]
    skills = [skill.strip() for skill in row["required_skills"].split(',')]
    job_skill_counts[job_title].update(skills)
rows = []
for job_title, skills in job_skill_counts.items():
    for skill, count in skills.items():
        rows.append({
            "job_title": job_title,
            "skill": skill,
            "count": count,
            "category": categorize(skill)
        })
skills_df = pd.DataFrame(rows)

job_titles = sorted(skills_df['job_title'].unique())
dropdown = widgets.Dropdown(options=job_titles, description='Job Title:', layout=widgets.Layout(width='50%'))
output = widgets.Output()

def plot_sorted_skills(job_title):
    with output:
        clear_output(wait=True)
        filtered = skills_df[skills_df['job_title'] == job_title].copy()
        filtered.sort_values(by='count', ascending=True, inplace=True)

        fig = px.bar(
            filtered,
            x='count',
            y='skill',
            color='category',
            orientation='h',
            title=f'Skill Counts for "{job_title}"',
            labels={'count': 'Count', 'skill': 'Skill', 'category': 'Category'},
            height=600,
            template='plotly_white',
            color_discrete_sequence=px.colors.qualitative.Set2
        )
        fig.update_layout(
            transition=dict(duration=600, easing='cubic-in-out'),
            margin=dict(l=120, r=40, t=60, b=40),
            legend_title_text='Skill Category',
            yaxis={'categoryorder': 'total ascending'}  # ensures sorting by count
        )
        fig.show()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        plot_sorted_skills(change['new'])

dropdown.observe(on_change)
display(dropdown, output)
plot_sorted_skills(dropdown.value)

Dropdown(description='Job Title:', layout=Layout(width='50%'), options=('AI Architect', 'AI Consultant', 'AI P…

Output()